In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

### Data preprocessing

In [55]:
def load_file(path):
    return pd.read_csv(path, delim_whitespace=True, header=None)
X_train=[]
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_acc_z_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/body_gyro_z_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_x_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_y_train.txt").to_numpy())
X_train.append(load_file("./UCI HAR dataset/UCI HAR dataset/train/Inertial Signals/total_acc_z_train.txt").to_numpy())
X_train=np.transpose(X_train, (1,2,0))

In [56]:
X_test=[]
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_acc_z_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/body_gyro_z_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_x_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_y_test.txt").to_numpy())
X_test.append(load_file("./UCI HAR dataset/UCI HAR dataset/test/Inertial Signals/total_acc_z_test.txt").to_numpy())
X_test=np.transpose(X_test, (1,2,0))

In [57]:
y_train=pd.read_csv("./UCI HAR dataset/UCI HAR dataset/train/y_train.txt", delim_whitespace=True, header=None)
y_test=pd.read_csv("./UCI HAR dataset/UCI HAR dataset/test/y_test.txt", delim_whitespace=True, header=None)

In [58]:
np.shape(y_train)

(7352, 1)

In [59]:
np.shape(X_train)

(7352, 128, 9)

In [60]:
activity_labels=pd.read_csv('./UCI HAR dataset/UCI HAR dataset/activity_labels.txt', delim_whitespace=True, header=None)

In [61]:
dict(activity_labels.values)

{1: 'WALKING',
 2: 'WALKING_UPSTAIRS',
 3: 'WALKING_DOWNSTAIRS',
 4: 'SITTING',
 5: 'STANDING',
 6: 'LAYING'}

In [62]:
y_train[0]=y_train[0].map(dict(activity_labels.values))
y_test[0]=y_test[0].map(dict(activity_labels.values))
y_train.columns=['Activity']
y_test.columns=['Activity']

In [64]:
y_train

,Activity
0,STANDING
1,STANDING
2,STANDING
3,STANDING
4,STANDING
...,...
7347,WALKING_UPSTAIRS
7348,WALKING_UPSTAIRS
7349,WALKING_UPSTAIRS
7350,WALKING_UPSTAIRS


In [65]:
encoder=OneHotEncoder(sparse_output=False)

In [66]:
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

In [67]:
type(y_train)

numpy.ndarray

In [68]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test=sc.fit_transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [69]:
np.shape(X_train)

(7352, 128, 9)

In [70]:
np.shape(X_test)

(2947, 128, 9)

### Building Model

In [71]:
n_timesteps, n_features, n_outputs=X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [72]:
n_outputs

6

In [73]:
n_timesteps

128

In [74]:
n_features

9

In [75]:
conv1d_model=tf.keras.Sequential()
conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features), padding='same'))
conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', padding='same'))
conv1d_model.add(tf.keras.layers.Dropout(rate=0.5))
conv1d_model.add(tf.keras.layers.MaxPooling1D(pool_size=1))
conv1d_model.add(tf.keras.layers.Flatten())
conv1d_model.add(tf.keras.layers.Dense(150, activation='relu'))
conv1d_model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))

conv1d_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
conv1d_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 128, 70)           1960      
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 70)           14770     
                                                                 
 dropout (Dropout)           (None, 128, 70)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 128, 70)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8960)              0         
                                                                 
 dense (Dense)               (None, 150)               1344150   
                                                        

In [77]:
training_history=conv1d_model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
230/230 [==============================] - 7s 6ms/step - loss: 0.2860 - accuracy: 0.8885
Epoch 2/50
230/230 [==============================] - 1s 4ms/step - loss: 0.1099 - accuracy: 0.9540
Epoch 3/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0944 - accuracy: 0.9595
Epoch 4/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0875 - accuracy: 0.9610
Epoch 5/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0810 - accuracy: 0.9663
Epoch 6/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0702 - accuracy: 0.9695
Epoch 7/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0720 - accuracy: 0.9676
Epoch 8/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0775 - accuracy: 0.9674
Epoch 9/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0543 - accuracy: 0.9751
Epoch 10/50
230/230 [==============================] - 1s 4ms/step - loss: 0.0569 - accuracy: 0.9758

In [78]:
_, accuracy=conv1d_model.evaluate(X_test, y_test, batch_size=32)

93/93 [==============================] - 1s 4ms/step - loss: 1.3827 - accuracy: 0.9230


In [79]:
accuracy

0.9229725003242493

In [80]:
def model_maker(X_train, y_train, X_test, y_test):
    n_timesteps, n_features, n_outputs=X_train.shape[1], X_train.shape[2], y_train.shape[1]

    conv1d_model=tf.keras.Sequential()
    conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features), padding='same'))
    conv1d_model.add(tf.keras.layers.Conv1D(filters=70, kernel_size=3, activation='relu', padding='same'))
    conv1d_model.add(tf.keras.layers.Dropout(rate=0.5))
    conv1d_model.add(tf.keras.layers.MaxPooling1D(pool_size=1))
    conv1d_model.add(tf.keras.layers.Flatten())
    conv1d_model.add(tf.keras.layers.Dense(150, activation='relu'))
    conv1d_model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))

    conv1d_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    training_history=conv1d_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    _, accuracy=conv1d_model.evaluate(X_test, y_test, batch_size=32)
    return accuracy

In [81]:
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [82]:
def experiment(num_of_repeats, X_train, y_train, X_test, y_test):
    scores = list()
    for n in range(num_of_repeats):
        score = model_maker(X_train, y_train, X_test, y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (n+1, score))
        scores.append(score)
    summarize_results(scores)

In [83]:
experiment(10, X_train, y_train, X_test, y_test)

93/93 [==============================] - 0s 3ms/step - loss: 0.8408 - accuracy: 0.9281
>#1: 92.806
93/93 [==============================] - 0s 3ms/step - loss: 0.8400 - accuracy: 0.9175
>#2: 91.754
93/93 [==============================] - 0s 3ms/step - loss: 1.4059 - accuracy: 0.9192
>#3: 91.924
93/93 [==============================] - 0s 3ms/step - loss: 0.8183 - accuracy: 0.9355
>#4: 93.553
93/93 [==============================] - 0s 3ms/step - loss: 0.9263 - accuracy: 0.9284
>#5: 92.840
93/93 [==============================] - 0s 3ms/step - loss: 1.1949 - accuracy: 0.9182
>#6: 91.822
93/93 [==============================] - 0s 3ms/step - loss: 0.9487 - accuracy: 0.9257
>#7: 92.569
93/93 [==============================] - 0s 3ms/step - loss: 0.8944 - accuracy: 0.9216
>#8: 92.162
93/93 [==============================] - 0s 3ms/step - loss: 1.4471 - accuracy: 0.9247
>#9: 92.467
93/93 [==============================] - 0s 2ms/step - loss: 0.7365 - accuracy: 0.9270
>#10: 92.704
[92.80624